In [1]:
import pandas as pd
import numpy as np

# -----------------------------
# 1. Setup
# -----------------------------
np.random.seed(42)
rows = 100000

# -----------------------------
# 2. Base Dimensions
# -----------------------------
data = {
    "date": pd.date_range(start="2025-01-01", periods=rows, freq="H"),

    "page_type": np.random.choice(
        ["Home", "Article", "Category"],
        rows,
        p=[0.3, 0.5, 0.2]
    ),

    "publisher_section": np.random.choice(
        ["News", "Sports", "Entertainment", "Tech"],
        rows,
        p=[0.35, 0.25, 0.25, 0.15]
    ),

    "device": np.random.choice(
        ["Mobile", "Desktop"],
        rows,
        p=[0.65, 0.35]
    ),

    "geography": np.random.choice(
        ["India", "US", "EU"],
        rows,
        p=[0.5, 0.3, 0.2]
    ),

    "ad_slot": np.random.choice(
        ["Top", "Mid", "Bottom"],
        rows,
        p=[0.4, 0.35, 0.25]
    ),

    "ad_format": np.random.choice(
        ["Banner", "Native", "Video"],
        rows,
        p=[0.45, 0.35, 0.2]
    )
}

df = pd.DataFrame(data)

# -----------------------------
# 3. Impressions (Supply)
# -----------------------------
df["impressions"] = np.random.randint(100, 2500, rows)

# -----------------------------
# 4. CTR Logic (Engagement)
# -----------------------------
base_ctr = np.where(df["device"] == "Mobile", 0.025, 0.018)

slot_bonus = np.where(df["ad_slot"] == "Top", 0.006,
               np.where(df["ad_slot"] == "Mid", 0.003, 0))

format_bonus = np.where(df["ad_format"] == "Video", 0.012,
                 np.where(df["ad_format"] == "Native", 0.006, 0))

page_bonus = np.where(df["page_type"] == "Article", 0.004, 0)

df["ctr"] = base_ctr + slot_bonus + format_bonus + page_bonus

df["clicks"] = (df["impressions"] * df["ctr"]).astype(int)

df.drop(columns=["ctr"], inplace=True)

# -----------------------------
# 5. Revenue Logic (Monetization)
# -----------------------------
geo_rate = df["geography"].map({
    "India": 0.45,
    "EU": 0.85,
    "US": 1.25
})

format_rate = df["ad_format"].map({
    "Banner": 0.8,
    "Native": 1.0,
    "Video": 1.6
})

device_rate = np.where(df["device"] == "Desktop", 1.25, 1.0)

section_rate = df["publisher_section"].map({
    "News": 1.0,
    "Sports": 1.1,
    "Entertainment": 1.2,
    "Tech": 1.35
})

df["revenue"] = (
    (df["impressions"] / 1000)
    * geo_rate
    * format_rate
    * device_rate
    * section_rate
    * np.random.uniform(0.85, 1.15, rows)
)

# -----------------------------
# 6. Save Dataset
# -----------------------------
df.to_csv("publisher_inventory_yield_data.csv", index=False)

# Preview
df.head()


C:\Users\Deep\AppData\Local\Temp\ipykernel_2352\3150280192.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  "date": pd.date_range(start="2025-01-01", periods=rows, freq="H"),


,date,page_type,publisher_section,device,geography,ad_slot,ad_format,impressions,clicks,revenue
0,2025-01-01 00:00:00,Article,Sports,Mobile,India,Bottom,Banner,1357,39,0.556550
1,2025-01-01 01:00:00,Category,Sports,Mobile,India,Top,Native,2043,75,1.152491
2,2025-01-01 02:00:00,Article,Sports,Mobile,India,Top,Native,1669,68,0.896667
3,2025-01-01 03:00:00,Article,Sports,Mobile,India,Bottom,Video,2481,101,1.768762
4,2025-01-01 04:00:00,Home,Sports,Mobile,EU,Mid,Native,1731,58,1.702391


In [3]:
import pandas as pd

df = pd.read_csv("publisher_inventory_yield_data.csv")
df.head()


,date,page_type,publisher_section,device,geography,ad_slot,ad_format,impressions,clicks,revenue
0,2025-01-01 00:00:00,Article,Sports,Mobile,India,Bottom,Banner,1357,39,0.556550
1,2025-01-01 01:00:00,Category,Sports,Mobile,India,Top,Native,2043,75,1.152491
2,2025-01-01 02:00:00,Article,Sports,Mobile,India,Top,Native,1669,68,0.896667
3,2025-01-01 03:00:00,Article,Sports,Mobile,India,Bottom,Video,2481,101,1.768762
4,2025-01-01 04:00:00,Home,Sports,Mobile,EU,Mid,Native,1731,58,1.702391


In [5]:
df.info()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   date               100000 non-null  object 
 1   page_type          100000 non-null  object 
 2   publisher_section  100000 non-null  object 
 3   device             100000 non-null  object 
 4   geography          100000 non-null  object 
 5   ad_slot            100000 non-null  object 
 6   ad_format          100000 non-null  object 
 7   impressions        100000 non-null  int64  
 8   clicks             100000 non-null  int64  
 9   revenue            100000 non-null  float64
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


date                 0
page_type            0
publisher_section    0
device               0
geography            0
ad_slot              0
ad_format            0
impressions          0
clicks               0
revenue              0
dtype: int64

In [25]:
df["CTR"] = df["clicks"] / df["impressions"]
df["RPM"] = (df["revenue"] / df["impressions"]) * 1000
df["eCPM"] = df["revenue"] / (df["impressions"] / 1000)
df[["CTR", "RPM", "eCPM"]].describe()

,CTR,RPM,eCPM
count,100000.000000,100000.000000,100000.000000
mean,0.031809,0.970556,0.970556
std,0.006571,0.571734,0.571734
min,0.009091,0.306017,0.306017
25%,0.027530,0.508628,0.508628
50%,0.031406,0.830925,0.830925
75%,0.036539,1.261847,1.261847
max,0.046999,3.879209,3.879209


In [27]:
overall_kpis = {
    "Total Impressions": df["impressions"].sum(),
    "Total Clicks": df["clicks"].sum(),
    "Overall CTR": df["clicks"].sum() / df["impressions"].sum(),
    "Total Revenue": df["revenue"].sum(),
    "Overall RPM": (df["revenue"].sum() / df["impressions"].sum()) * 1000
}

overall_kpis


{'Total Impressions': 129844905,
 'Total Clicks': 4168660,
 'Overall CTR': 0.03210491778633902,
 'Total Revenue': 126021.9526440033,
 'Overall RPM': 0.9705575482072499}

In [ ]:
Segment analysis

In [31]:
device_perf = (
    df.groupby("device")
    .agg(
        impressions=("impressions", "sum"),
        clicks=("clicks", "sum"),
        revenue=("revenue", "sum")
    )
)

device_perf["CTR"] = device_perf["clicks"] / device_perf["impressions"]
device_perf["RPM"] = (device_perf["revenue"] / device_perf["impressions"]) * 1000

device_perf


,impressions,clicks,revenue,CTR,RPM
device,,,,,
Desktop,45349946,1247586,50615.715678,0.027510,1.116114
Mobile,84494959,2921074,75406.236966,0.034571,0.892435


In [33]:
format_perf = (
    df.groupby("ad_format")
    .agg(
        impressions=("impressions", "sum"),
        revenue=("revenue", "sum")
    )
)

format_perf["RPM"] = (format_perf["revenue"] / format_perf["impressions"]) * 1000
format_perf.sort_values("RPM", ascending=False)


,impressions,revenue,RPM
ad_format,,,
Video,25755503,38767.623508,1.505217
Native,45709522,43307.224713,0.947444
Banner,58379880,43947.104423,0.752778


In [35]:
slot_perf = (
    df.groupby("ad_slot")
    .agg(
        impressions=("impressions", "sum"),
        revenue=("revenue", "sum")
    )
)

slot_perf["RPM"] = (slot_perf["revenue"] / slot_perf["impressions"]) * 1000
slot_perf


,impressions,revenue,RPM
ad_slot,,,
Bottom,32414637,31307.073562,0.965831
Mid,45878252,44753.181493,0.975477
Top,51552016,49961.697589,0.969151


In [37]:
section_perf = (
    df.groupby("publisher_section")
    .agg(
        impressions=("impressions", "sum"),
        revenue=("revenue", "sum")
    )
)

section_perf["RPM"] = (section_perf["revenue"] / section_perf["impressions"]) * 1000
section_perf.sort_values("RPM", ascending=False)


,impressions,revenue,RPM
publisher_section,,,
Tech,19658527,22721.323688,1.155800
Entertainment,32513148,33598.165154,1.033372
Sports,32541739,30915.120473,0.950014
News,45131491,38787.343329,0.859430


In [39]:
overall_rpm = overall_kpis["Overall RPM"]

low_yield = (
    df.groupby(["page_type", "device", "ad_slot", "ad_format"])
    .agg(
        impressions=("impressions", "sum"),
        revenue=("revenue", "sum")
    )
    .reset_index()
)

low_yield["RPM"] = (low_yield["revenue"] / low_yield["impressions"]) * 1000
low_yield["Yield_Gap"] = low_yield["RPM"] - overall_rpm

low_yield.sort_values("Yield_Gap").head(10)


,page_type,device,ad_slot,ad_format,impressions,revenue,RPM,Yield_Gap
27,Category,Mobile,Bottom,Banner,1896951,1291.197409,0.680670,-0.289888
33,Category,Mobile,Top,Banner,3043888,2084.087050,0.684679,-0.285878
9,Article,Mobile,Bottom,Banner,4771677,3278.600916,0.687096,-0.283461
15,Article,Mobile,Top,Banner,7461310,5129.435000,0.687471,-0.283086
12,Article,Mobile,Mid,Banner,6722031,4652.749325,0.692164,-0.278393
48,Home,Mobile,Mid,Banner,4011783,2780.199093,0.693008,-0.277549
51,Home,Mobile,Top,Banner,4474333,3133.001802,0.700217,-0.270341
45,Home,Mobile,Bottom,Banner,2843906,1992.584207,0.700651,-0.269907
30,Category,Mobile,Mid,Banner,2698538,1899.459311,0.703885,-0.266673
21,Category,Desktop,Mid,Banner,1413456,1185.664812,0.838841,-0.131717
